In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/rmgpy/test_data'}

In [3]:
thermo_libs = [
'C1_C2_Fluorine', 
'NCSU_C2_C8_PFAS', 
'PFCA_thermo',
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ['1+2_Cycloaddition', 'default', 'Disproportionation-Y']


#'R_Recombination', #no problem
#'1,2_Insertion_carbene', #no problem
#'F_Abstraction', #no problem
#'R_Addition_MultipleBond', #no problem, 
#'XY_Addition_MultipleBond', #no problem, 
#'Singlet_Carbene_Intra_Disproportionation', 
#'Intra_R_Add_Endocyclic', 'R_Addition_COm', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement']


In [4]:
#try 1
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = ['halogens_pdep'],
            seed_mechanisms = ['FFCM1(-)'],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            depository = True, 
        )


[<rmgpy.molecule.group.Group object at 0x2b94ef374f30>, <rmgpy.molecule.group.Group object at 0x2b94ef61d590>]
[<rmgpy.molecule.group.Group object at 0x2b94ef280980>, <rmgpy.molecule.group.Group object at 0x2b94ef61d590>]
[<rmgpy.molecule.group.Group object at 0x2b94ef280c90>, <rmgpy.molecule.group.Group object at 0x2b94ef61d590>]
[<rmgpy.molecule.group.Group object at 0x2b94ef449ec0>, <rmgpy.molecule.group.Group object at 0x2b94ef61d590>]
[<rmgpy.molecule.group.Group object at 0x2b94ef449d70>, <rmgpy.molecule.group.Group object at 0x2b94ef61d590>]
[<rmgpy.molecule.group.Group object at 0x2b94ef61d4b0>, <rmgpy.molecule.group.Group object at 0x2b94ef61d590>]
[<rmgpy.molecule.group.Group object at 0x2b94ef61d520>, <rmgpy.molecule.group.Group object at 0x2b94ef61d590>]
[<rmgpy.molecule.group.Group object at 0x2b94ef374f30>, <rmgpy.molecule.group.Group object at 0x2b94ef61d9f0>]
[<rmgpy.molecule.group.Group object at 0x2b94ef280980>, <rmgpy.molecule.group.Group object at 0x2b94ef61d9f0>]
[

In [5]:
database.thermo

In [6]:
database.kinetics.families['1+2_Cycloaddition'].add_rules_from_training(thermo_database=database.thermo)

1+2_Cycloaddition


ERROR:root:Could not update atomtypes for this molecule:
multiplicity -187
1 *1 O u0 p2 c0 {3,S} {4,S}
2    O u0 p2 c0 {3,D}
3 *2 C u0 p0 c0 {1,S} {2,D} {4,S}
4 *3 F u0 p2 c0 {1,S} {3,S} {6,S}
5    F u0 p3 c0 {6,S}
6    C u0 p1 c0 {4,S} {5,S}



[Molecule(smiles="O=C=O"), Molecule(smiles="F[C]F")]


AtomTypeError: Unable to determine atom type for atom F, which has 3 single bonds, 0 double bonds (0 to O, 0 to S, 0 others), 0 triple bonds, 0 quadruple bonds, 0 benzene bonds, 2 lone pairs, and +0 charge.

In [ ]:
import os

In [ ]:
kin_families = ['R_Recombination', '1,2_Insertion_carbene', 'F_Abstraction', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', 'Intra_R_Add_Endocyclic', 'R_Addition_COm', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement']


In [ ]:
for fam in kin_families:
    input_string = f'''
thermolibs = [
'C1_C2_Fluorine', 
'NCSU_C2_C8_PFAS', 
'PFCA_thermo',
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

database(
thermoLibraries = thermolibs,
reactionLibraries = ['halogens_pdep'], 
seedMechanisms= ['FFCM1(-)'],
kineticsDepositories = ['training'],
kineticsFamilies = ['default','{fam}','Disproportionation-Y'], #default and halogens incorporates new H and F abstraction rates
frequenciesLibraries = ['halogens_G4'],
kineticsEstimator = 'rate rules',
)
'''
    rest_of_string = '''
species(
    label = '2,3,3,3-tetrafluoropropene',
    reactive = True,
    structure = SMILES('C=C(C(F)(F)F)F')
)

species(
    label = 'OH',
    reactive = True,
    structure = SMILES('[OH]')
)

    
species(
    label = 'O2',
    reactive = True,
    structure = SMILES('[O][O]')
)
    
species(
    label = 'H2O',
    reactive = True,
    structure = SMILES('O')
)
    
species(
    label = 'N2',
    reactive = False,
    structure = SMILES('N#N')
)
    

simpleReactor(
        temperature=[(1000,'K'),(2000,'K')],
        pressure= [(1.0,'bar'),(10.0, 'bar')],
        nSims=12,
        initialMoleFractions={
        "2,3,3,3-tetrafluoropropene" : [0.2,1.0],
        "O2": 1,
        "N2": 3.76,
        },
        terminationConversion={
        '2,3,3,3-tetrafluoropropene': 0.999,
        },
        #terminationRateRatio=1e-4,
        #terminationTime=(10,'s'),
        terminationTime=(1,'s'),
        sensitivity=['2,3,3,3-tetrafluoropropene','OH'],
        sensitivityThreshold=0.001,
        )
        
        
model(
    toleranceMoveToCore = 0.1,
    toleranceInterruptSimulation = 0.1,
    maximumEdgeSpecies = 3e5,
    filterReactions = True,
    filterThreshold = 5e8,
    minCoreSizeForPrune = 50,
    minSpeciesExistIterationsForPrune = 4,
)

pressureDependence(
    method='modified strong collision',
    maximumGrainSize=(0.5,'kcal/mol'),
    minimumNumberOfGrains=250,
    temperatures=(300,2500,'K',8),
    pressures=(0.01,100,'bar',5),
    interpolation=('Chebyshev', 6, 4),
    maximumAtoms=16,
)

simulator(
    atol = 1e-16,
    rtol = 1e-08,
    sens_atol = 1e-06,
    sens_rtol = 0.0001,
)

generatedSpeciesConstraints(
    allowed=['input species','seed mechanisms','reaction libraries'],
    maximumCarbonAtoms=8,
    maximumOxygenAtoms=6,
    maximumRadicalElectrons=2,
    maximumSingletCarbenes=1,
    maximumCarbeneRadicals=0,
    allowSingletO2 = True,
)

options(
    units = "si",
    generateSeedEachIteration = True,
    generateOutputHTML = True,
    generatePlots = True,
    saveSimulationProfiles = True,
    saveEdgeSpecies = True,
    keepIrreversible = True,
    verboseComments = False,
)
'''
    my_file_name = f'./{fam}/input.py'
    with open(my_file_name, 'w') as f:
        f.write(input_string+rest_of_string)
        
    run_string = f'''
#!/bin/sh

#SBATCH --nodes=1
#SBATCH --time=1-00:00:00
#SBATCH --job-name={fam}
#SBATCH --error={fam}.slurm.log
#SBATCH --cpus-per-task=5
#SBATCH --mem-per-cpu=8Gb
#SBATCH --ntasks=1 
#SBATCH --array=1
#SBATCH --partition=short

python-jl /home/khalil.nor/Code/RMG-Py/rmg.py input.py
 
'''
    my_file_name = f'./{fam}/run.sh'
    with open(my_file_name, 'w') as f:
        f.write(run_string)


In [ ]:
os.chdir('/work/westgroup/nora/Code/projects/PFAS/AIChE_2024/models_after_retraining/retrained_all_but_R_recombination/debugging_load_error')